In [56]:
import matplotlib.pyplot as plt
%matplotlib inline
import IPython.display
import librosa.display
import numpy as np
import librosa
import tensorflow as tf
import glob

In [57]:
import glob
#uav_path = 'C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/JUNE_01_PHANTOMS/*.wav'
#none_path = 'C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/JUNE_02_BACKGROUND/background_0*.wav'
#uav_path = '../../../0720/*.wav'
#uav_path = 'C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/Sounddata_new/22050/Drone_22050/*.*'
#none_path = 'C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/Sounddata_new/22050/None_22050/*.*'



uav_path = '/Users/junhyuckwoo/purdueProject/uav-audio-detection/soundData/0720A/load/*.*'
loaded_path = '/Users/junhyuckwoo/purdueProject/uav-audio-detection/soundData/0720A/unload/*.*'
none_path = '/Users/junhyuckwoo/purdueProject/uav-audio-detection/soundData/BACKGROUND/background_0*.wav'


uav_files = glob.glob(uav_path)
loaded_files = glob.glob(loaded_path)
none_files = glob.glob(none_path)


In [58]:
CHUNK_SIZE = 8192
SR = 22050
N_MFCC = 16

In [59]:
def load(files, sr=22050):
    [raw, sr] = librosa.load(files[0], sr=sr)
    for f in files[1:]:
        [array, sr] = librosa.load(f, sr=sr)
        raw = np.hstack((raw, array))
    print(raw.shape)
    return raw


In [60]:
'''
from myAudio import Audio
import pyaudio
import time
import wave
import numpy as np
import librosa
def load2(sr=22050):
    raw = Audio.getStream(sample_rate = sr, chunk_size = CHUNK_SIZE,chunk_num = 1, isWrite=False)
    raw = np.array(raw)
    print(raw.shape)
    return raw
sound_raw = load2()
'''

'\nfrom myAudio import Audio\nimport pyaudio\nimport time\nimport wave\nimport numpy as np\nimport librosa\ndef load2(sr=22050):\n    raw = Audio.getStream(sample_rate = sr, chunk_size = CHUNK_SIZE,chunk_num = 1, isWrite=False)\n    raw = np.array(raw)\n    print(raw.shape)\n    return raw\nsound_raw = load2()\n'

In [61]:
uav_raw = load(uav_files)
loaded_raw = load(loaded_files)
none_raw = load(none_files)

(9064894,)
(12584212,)
(9289728,)


# Data preprocessing

Chose `mfcc4` 

In [62]:
def mfcc4(raw, label, chunk_size=8192, window_size=4096, sr=22050, n_mfcc=16, n_frame=16):
    mfcc = np.empty((0, n_mfcc, n_frame))
    y = []
    print(raw.shape)
    for i in range(0, len(raw), chunk_size//2):
        mfcc_slice = librosa.feature.mfcc(raw[i:i+chunk_size], sr=sr, n_mfcc=n_mfcc) #n_mfcc,17
        if mfcc_slice.shape[1] < 17:
            print("small end:", mfcc_slice.shape)
            continue
        mfcc_slice = mfcc_slice[:,:-1]
        mfcc_slice = mfcc_slice.reshape((1, mfcc_slice.shape[0], mfcc_slice.shape[1]))
        mfcc = np.vstack((mfcc, mfcc_slice))
        y.append(label)
    y = np.array(y)
    return mfcc, y

In [63]:
mfcc_loaded, y_loaded = mfcc4(loaded_raw, 2)
mfcc_uav, y_uav = mfcc4(uav_raw, 1)
mfcc_none, y_none = mfcc4(none_raw, 0)

print(mfcc_uav.shape, y_uav.shape)
print(mfcc_loaded.shape, y_loaded.shape)
print(mfcc_none.shape, y_none.shape)

(12584212,)
small end: (16, 11)
small end: (16, 3)
(9064894,)
small end: (16, 9)
small end: (16, 1)
(9289728,)
small end: (16, 9)
(2212, 16, 16) (2212,)
(3071, 16, 16) (3071,)
(2267, 16, 16) (2267,)


In [64]:
X = np.concatenate((mfcc_loaded, mfcc_uav, mfcc_none), axis=0)
y = np.hstack((y_loaded, y_uav, y_none))
print(X.shape, y.shape)

(7550, 16, 16) (7550,)


In [65]:
X = np.reshape(X,(X.shape[0],-1)) #reshape

In [66]:
X.shape

(7550, 256)

In [67]:
n_labels = y.shape[0]
n_unique_labels = 3
y_encoded = np.zeros((n_labels, n_unique_labels))
y_encoded[np.arange(n_labels), y] = 1
print(y_encoded.shape)

(7550, 3)


In [68]:
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [69]:
X_train, X_val, y_train, y_val = model_selection.train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [70]:
print(X_train.shape,y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(4832, 256) (4832, 3)
(1208, 256) (1208, 3)
(1510, 256) (1510, 3)


In [71]:
### path selection between jaesung & gunhoo ###
np.save('/Users/junhyuckwoo/purdueProject/uav-audio-detection/models/JUN/Dense/data/X_train_chunk_3d', X_train)
np.save('/Users/junhyuckwoo/purdueProject/uav-audio-detection/models/JUN/Dense/data/X_test_chunk_3d', X_test)
np.save('/Users/junhyuckwoo/purdueProject/uav-audio-detection/models/JUN/Dense/data/y_train_chunk_3d', y_train)
np.save('/Users/junhyuckwoo/purdueProject/uav-audio-detection/models/JUN/Dense/data/y_test_chunk_3d', y_test)
np.save('/Users/junhyuckwoo/purdueProject/uav-audio-detection/models/JUN/Dense/data/X_test_chunk_3d', X_test)
np.save('/Users/junhyuckwoo/purdueProject/uav-audio-detection/models/JUN/Dense/data/y_val_chunk_3d', y_val)
np.save('/Users/junhyuckwoo/purdueProject/uav-audio-detection/models/JUN/Dense/data/X_val_chunk_3d', X_val)
#np.save('../../../RW_AUDIO_DATA_2018_Update/Xy/X_train_chunk_3d', X_train)
#np.save('../../../RW_AUDIO_DATA_2018_Update/Xy/X_test_chunk_3d', X_test)
#np.save('../../../RW_AUDIO_DATA_2018_Update/Xy/y_train_chunk_3d', y_train)
#np.save('../../../RW_AUDIO_DATA_2018_Update/Xy/y_test_chunk_3d', y_test)

In [72]:
### path selection between jaesung & gunhoo ###
X_train = np.load('/Users/junhyuckwoo/purdueProject/uav-audio-detection/models/JUN/Dense/data/X_train_chunk_3d.npy')
X_test = np.load('/Users/junhyuckwoo/purdueProject/uav-audio-detection/models/JUN/Dense/data/X_test_chunk_3d.npy')
y_train = np.load('/Users/junhyuckwoo/purdueProject/uav-audio-detection/models/JUN/Dense/data/y_train_chunk_3d.npy')
y_test = np.load('/Users/junhyuckwoo/purdueProject/uav-audio-detection/models/JUN/Dense/data/y_test_chunk_3d.npy')
y_val = np.load('/Users/junhyuckwoo/purdueProject/uav-audio-detection/models/JUN/Dense/data/y_val_chunk_3d.npy')
X_val = np.load('/Users/junhyuckwoo/purdueProject/uav-audio-detection/models/JUN/Dense/data/X_val_chunk_3d.npy')

#X_train = np.load('../../../RW_AUDIO_DATA_2018_Update/Xy/X_train_chunk_3d.npy')
#X_test = np.load('../../../RW_AUDIO_DATA_2018_Update/Xy/X_test_chunk_3d.npy')
#y_train = np.load('../../../RW_AUDIO_DATA_2018_Update/Xy/y_train_chunk_3d.npy')
#y_test = np.load('../../../RW_AUDIO_DATA_2018_Update/Xy/y_test_chunk_3d.npy')

# Dense Experiment


In [73]:
tf.reset_default_graph()

In [74]:
n_mfcc = 16
n_frame = 16
n_classes = 3
n_channels = 1

learning_rate = 0.0002  # 0.005


## Experiment 
- learning rate 0.0002
- dense with dropout / five layers
- multiclass(3) classifier model
- Loaded UAV / Unloaded UAV / Background noise

In [75]:
X = tf.placeholder(tf.float32, shape=[None,n_mfcc*n_frame*n_channels])
#X = tf.reshape(X, [-1, n_mfcc, n_frame, n_channels]) #fully connected에는 리셰입 안해도 된다 
Y = tf.placeholder(tf.float32, shape=[None,n_classes])

keep_prob = tf.placeholder(tf.float32)

#dense2 = tf.layers.dense(inputs=flat, units=625, activation=tf.nn.relu)
#dropout2 = tf.layers.dropout(inputs=dense2, rate=0.5, training=True)

dense1 = tf.layers.dense(inputs=X, units=256, activation=tf.nn.relu)
dropout1 = tf.nn.dropout(dense1, keep_prob=keep_prob)


dense2 = tf.layers.dense(inputs=dropout1, units=256, activation=tf.nn.relu)
dropout2 = tf.nn.dropout(dense2, keep_prob=keep_prob)

dense3 = tf.layers.dense(inputs=dropout2, units=512, activation=tf.nn.relu)
dropout3 = tf.nn.dropout(dense3, keep_prob=keep_prob)

#이거 지워보고 돌려보고 
dense4 = tf.layers.dense(inputs=dropout3, units=512, activation=tf.nn.relu)
dropout4 = tf.nn.dropout(dense4, keep_prob=keep_prob)


dense5 = tf.layers.dense(inputs=dropout4, units=256, activation=tf.nn.relu)
dropout5 = tf.nn.dropout(dense5, keep_prob=keep_prob)


logits= tf.layers.dense(inputs=dropout5, units=3)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [76]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [77]:
model_path = '/Users/junhyuckwoo/purdueProject/uav-audio-detection/models/JUN/Dense/my_test_model_cnn'
#model_path = '../models/CNN/my_test_model_cnn'
#saver = tf.train.Saver()
#model_path = './model_cnn.ckpt'
#model_path = '../my_test_model_cnn'
saver = tf.train.Saver()

In [78]:
import math

In [79]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
#    accuracy_summary = tf.summary.scalar('accuracy', accuracy)

In [80]:
from sklearn.metrics import accuracy_score

In [81]:
#original 
batch_size = 64
cost_history = np.empty(shape=[1], dtype=float)

for epoch in range(1,50):
    avg_cost = 0    
#    total_batch = int(X_train.shape[0] / batch_size)
    for i in range( int(math.ceil(len(X_train)/batch_size)) ): # 배치 사이즈로 나눈 나머지 다 돌 수 있게 
        x_ = X_train[batch_size * i: batch_size * (i + 1)]
        y_ = y_train[batch_size * i: batch_size * (i + 1)]
        feed_dict={X:x_, Y:y_, keep_prob:0.6}   
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        cost_history = np.append(cost_history,cost)
        avg_cost += c
    #accuracy_val = sess.run([accuracy], feed_dict={X:X_val, Y:y_val
    val_pred = sess.run(tf.argmax(logits,1),feed_dict={X: X_val , keep_prob:1})  
    val_true = sess.run(tf.argmax(y_val,1))
    accuracy_val = accuracy_score(val_pred, val_true)
    print('Epoch:', '%04d' % (epoch), 'cost = ', '{:.9f}'.format(avg_cost / len(X_train)), "\t검증 세트 정확도: {:.3f}%".format(accuracy_val * 100),)
saver.save(sess, model_path)

#if epoch% 20 == 0:
#   save

Epoch: 0001 cost =  0.441206775 	검증 세트 정확도: 55.050%
Epoch: 0002 cost =  0.112008268 	검증 세트 정확도: 62.997%
Epoch: 0003 cost =  0.048526676 	검증 세트 정확도: 65.315%
Epoch: 0004 cost =  0.027489776 	검증 세트 정확도: 67.798%
Epoch: 0005 cost =  0.018115631 	검증 세트 정확도: 64.156%
Epoch: 0006 cost =  0.014615816 	검증 세트 정확도: 76.656%
Epoch: 0007 cost =  0.011323070 	검증 세트 정확도: 93.709%
Epoch: 0008 cost =  0.010124053 	검증 세트 정확도: 94.702%
Epoch: 0009 cost =  0.008754129 	검증 세트 정확도: 95.116%
Epoch: 0010 cost =  0.007751005 	검증 세트 정확도: 96.109%
Epoch: 0011 cost =  0.006643001 	검증 세트 정확도: 96.358%
Epoch: 0012 cost =  0.005690227 	검증 세트 정확도: 96.440%
Epoch: 0013 cost =  0.004871955 	검증 세트 정확도: 96.689%
Epoch: 0014 cost =  0.003914089 	검증 세트 정확도: 96.689%
Epoch: 0015 cost =  0.003747035 	검증 세트 정확도: 96.689%
Epoch: 0016 cost =  0.003229421 	검증 세트 정확도: 96.606%
Epoch: 0017 cost =  0.003029937 	검증 세트 정확도: 96.854%
Epoch: 0018 cost =  0.002799868 	검증 세트 정확도: 96.937%
Epoch: 0019 cost =  0.002771885 	검증 세트 정확도: 96.937%
Epoch: 0020 

'/Users/junhyuckwoo/purdueProject/uav-audio-detection/models/JUN/Dense/my_test_model_cnn'

In [82]:
y_pred = sess.run(tf.argmax(logits,1),feed_dict={X: X_test , keep_prob:1}) # keep_prob 삭제  
y_true = sess.run(tf.argmax(y_test,1))

In [83]:
from sklearn.metrics import precision_recall_fscore_support
p,r,f,s = precision_recall_fscore_support(y_true, y_pred, average='micro')
print("F-Score:", round(f,3))
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(y_true, y_pred))

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true, y_pred))

F-Score: 0.98
Accuracy:  0.9801324503311258
             precision    recall  f1-score   support

          0       1.00      1.00      1.00       449
          1       0.95      0.98      0.96       422
          2       0.99      0.97      0.98       639

avg / total       0.98      0.98      0.98      1510

[[449   0   0]
 [  0 413   9]
 [  0  21 618]]
